In [25]:
#import eveerything that needed
import re
import pandas as pd
import datetime
import plotly
import plotly.plotly as py
import plotly.graph_objs as go 
plotly.tools.set_credentials_file(username='FangzhengZhang', api_key='JCzdvumGOr8rCKHaeeeK')

In [3]:
#IO data 
table = pd.read_csv('employee_reviews.csv')
table

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...
2,3,google,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,5.0,4.0,5.0,5.0,4.0,949,https://www.glassdoor.com/Reviews/Google-Revie...
3,4,google,"Mountain View, CA","Feb 8, 2015",Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5.0,2.0,5.0,5.0,4.0,5.0,498,https://www.glassdoor.com/Reviews/Google-Revie...
4,5,google,"Los Angeles, CA","Jul 19, 2018",Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5.0,5.0,5.0,5.0,5.0,5.0,49,https://www.glassdoor.com/Reviews/Google-Revie...
5,6,google,"Mountain View, CA","Dec 9, 2018",Former Employee - SDE2,NICE working in GOOGLE as an INTERN,"People are not that busy, so they are nice to ...",Food is not good as I expected. People said it...,none,5.0,4.0,4.0,4.0,5.0,4.0,1,https://www.glassdoor.com/Reviews/Google-Revie...
6,7,google,"New York, NY","Dec 11, 2018",Current Employee - Software Engineer,Software engineer,Great working environment. Good work life balance,Usual big company problems. Hierarchy.,none,5.0,5.0,4.0,4.0,5.0,4.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
7,8,google,none,"Dec 11, 2018",Former Employee - Anonymous Employee,great place to work and progress,"work culture, benefits, growth, people,",No cons that i can think of,none,5.0,5.0,5.0,5.0,5.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
8,9,google,"New York, NY","Dec 10, 2018",Current Employee - Anonymous Employee,Google Surpasses Realistic Expectations,Great products. Vision you can feel good about...,Younger employees complaining about the compan...,none,5.0,5.0,5.0,5.0,5.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
9,10,google,none,"Dec 9, 2018",Current Employee - Anonymous Employee,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,none,5.0,5.0,5.0,5.0,5.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...


In [4]:
#print all the important info
company_set = set(table.company)
amazon_table = table[table.company == 'amazon']
google_table = table[table.company == 'google']
apple_table = table[table.company == 'apple']
facebook_table = table[table.company == 'facebook']
microsoft_table = table[table.company == 'microsoft']
netflix_table = table[table.company == 'netflix']

In [5]:
#plot the number of reviews for each company 
review_count_dic = {}
review_count_dic['amazon'] = len(amazon_table)
review_count_dic['apple'] = len(apple_table) 
review_count_dic['facebook'] = len(facebook_table)
review_count_dic['google'] = len(google_table)
review_count_dic['microsoft'] = len(microsoft_table)
review_count_dic['netflix'] = len(netflix_table)


In [6]:
#plot the number of review for each company
count_table_data = [go.Bar(
                x=list(review_count_dic.keys()),
                y= list(review_count_dic.values()))]
py.iplot(count_table_data, filename='basic-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~FangzhengZhang/0 or inside your plot.ly account where it is named 'basic-bar'


c:\users\fangz\appdata\local\programs\python\python37\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [45]:
date_look_up_dic = {"Jan": 1 ,"Feb": 2 ,"Mar": 3 ,"Apr": 4, "May": 5, "Jun": 6,"Jul": 7,"Aug": 8,"Sep": 9,"Oct": 10,"Nov": 11,"Dec": 12}
def change_date_format(in_str):
    '''Dec 11, 2018  ->  2018 12 11'''
    str_part = re.split(' |, ',in_str)
    r_str = ''+ str_part[3] + '-' + str(date_look_up_dic[str_part[1]]) + '-' + str_part[2]
    try:
        date_time_obj = datetime.datetime.strptime(r_str, '%Y-%m-%d')
    except ValueError:
        date_time_obj = datetime.datetime.strptime("6666-6-6", '%Y-%m-%d')
    date_time_obj = date_time_obj.date()
    return date_time_obj
    
def clear_date_none(temp_table):
    return_table = temp_table[temp_table.dates != 'None']
    #todo change the date format 
    return_table.dates = return_table.dates.map(change_date_format)
    return_table = return_table.sort_values(by='dates', ascending=False)
    return return_table

In [46]:
#delete
r = map(change_date_format,amazon_table)
print (r)

In [47]:
#clean the tables and sort them by date
amazon_date_clean_table = clear_date_none(amazon_table)
apple_date_clean_table = clear_date_none(apple_table)
facebook_date_clean_table = clear_date_none(facebook_table)
google_date_clean_table = clear_date_none(google_table)
microsoft_date_clean_table = clear_date_none(microsoft_table)
netflix_date_clean_table = clear_date_none(netflix_table)

amazon_date_clean_table

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
28729,28730,amazon,Hyderabad (India),6666-06-06,Current Employee - NOC,Life in AMAZON,Excellent work culture No dress code Good CAB ...,No flexible Timings (Rotational shifts),none,4.0,3.0,4.0,4.0,3.0,4.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
7906,7907,amazon,none,2018-12-11,Former Employee - Anonymous Employee,Wild company,Learn a lot of things. Move fast.,You have to be really aggressive. The fittest ...,none,3.0,1.0,3.0,3.0,1.0,3.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
25451,25452,amazon,"Brampton, ON (Canada)",2018-12-11,Current Employee - Ex Employee,"Very busy, Hard working, demanding",Can be flexible regarding you to taking holida...,"It is full of favoritism, no equal opportuniti...",Not going to listen you.,2.0,1.0,1.0,2.0,1.0,1.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
25346,25347,amazon,Chennai (India),2018-12-11,Current Employee - Language Specialist,German language associate,less work stress at workplace,Growth can be slow depends on your own.,none,4.0,4.0,5.0,3.0,3.0,3.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
25345,25346,amazon,"Hemel Hempstead, England (UK)",2018-12-11,Former Employee - Warehouse Associate,Warehouse,free drinks and free paarkimg so its good,long hours and 10hr shifts,none,4.0,2.0,1.0,1.0,3.0,1.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
7831,7832,amazon,none,2018-12-11,Current Employee - Anonymous Employee,Work hard and you’ll go far,Great people and room to show your skills on a...,"Work/life balance, pay for the amount of work",none,4.0,2.0,5.0,3.0,4.0,4.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
7830,7831,amazon,none,2018-12-11,Current Employee - Anonymous Employee,On road driver assistant,Decent pay. Challenging work. Great co workers,I work 4 days 10 hour shifts.,none,4.0,3.0,4.0,4.0,4.0,4.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
7829,7830,amazon,none,2018-12-11,Current Employee - Warehouse Sortation Associate,Warehouse Sortation Associate,- Company works like a well oiled machine - Ma...,- A lot of the workers don't really care and h...,"Don't just hire everyone, put a bit more effor...",4.0,5.0,4.0,3.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
8119,8120,amazon,"Carlsbad, CA",2018-12-11,Former Employee - Sorter,Ok Starter Job,"Predictable schedule, No big curveballs",When change happens it tends to throw everybod...,Listen to workers opinions more,1.0,1.0,1.0,1.0,1.0,1.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...
25447,25448,amazon,Bengaluru (India),2018-12-11,Former Employee - Software Engineer,Good place to work in,"Competitive environment, payment is justified","Too much workload, not free even during weekends",Lessen the workload so that some time left for...,3.0,3.0,3.0,3.0,3.0,3.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...


In [62]:
def count_dates(temp_table):
    temp_time_dic = {}
    for i in range(len(temp_table)):
        row = temp_table.iloc[i]
        if str(row.dates) not in temp_time_dic:
            temp_time_dic[str(row.dates)] = 1
        else:
            temp_time_dic[str(row.dates)]+=1
    return temp_time_dic

In [63]:
#big file, take long time to run!!!!
#count the time line 
amazon_time_line_dic = count_dates(amazon_date_clean_table)
apple_time_line_dic = count_dates(apple_date_clean_table)
facebook_time_line_dic = count_dates(facebook_date_clean_table)
google_time_line_dic = count_dates(google_date_clean_table)
microsoft_time_line_dic = count_dates(microsoft_date_clean_table)
netflix_time_line_dic = count_dates(netflix_date_clean_table)


In [64]:
netflix_time_line_dic


{'6666-06-06': 1,
 '2018-12-10': 1,
 '2018-12-05': 1,
 '2018-12-03': 1,
 '2018-11-30': 1,
 '2018-11-26': 1,
 '2018-11-20': 1,
 '2018-11-18': 1,
 '2018-11-08': 1,
 '2018-11-03': 1,
 '2018-10-31': 1,
 '2018-10-29': 1,
 '2018-10-27': 1,
 '2018-10-26': 1,
 '2018-10-22': 2,
 '2018-10-21': 1,
 '2018-10-20': 1,
 '2018-10-18': 1,
 '2018-10-17': 1,
 '2018-10-15': 1,
 '2018-10-14': 1,
 '2018-10-11': 3,
 '2018-10-10': 1,
 '2018-10-08': 1,
 '2018-10-05': 1,
 '2018-10-04': 1,
 '2018-10-02': 1,
 '2018-09-29': 2,
 '2018-09-26': 1,
 '2018-09-20': 2,
 '2018-09-17': 2,
 '2018-09-11': 1,
 '2018-09-09': 1,
 '2018-09-06': 1,
 '2018-08-31': 1,
 '2018-08-30': 1,
 '2018-08-27': 1,
 '2018-08-22': 1,
 '2018-08-16': 1,
 '2018-08-15': 1,
 '2018-08-12': 1,
 '2018-08-09': 1,
 '2018-08-04': 1,
 '2018-07-31': 1,
 '2018-07-26': 1,
 '2018-07-17': 1,
 '2018-07-16': 1,
 '2018-07-13': 1,
 '2018-07-11': 1,
 '2018-07-07': 1,
 '2018-07-01': 1,
 '2018-06-26': 2,
 '2018-06-23': 1,
 '2018-06-15': 1,
 '2018-06-11': 1,
 '2018-06-

In [67]:
#plot time line
amazon_time_line = go.Scatter(
                x=list(amazon_time_line_dic.keys()),
                y=list(amazon_time_line_dic.values()),
                name = "Amazon Time Line",
                line = dict(color = '#f44542'),
                opacity = 0.8)
apple_time_line = go.Scatter(
                x=list(apple_time_line_dic.keys()),
                y=list(apple_time_line_dic.values()),
                name = "Apple Time Line",
                line = dict(color = '#f4f141'),
                opacity = 0.8)
facebook_time_line = go.Scatter(
                x=list(facebook_time_line_dic.keys()),
                y=list(facebook_time_line_dic.values()),
                name = "Facebook Time Line",
                line = dict(color = '#41f45e'),
                opacity = 0.8)
google_time_line = go.Scatter(
                x=list(google_time_line_dic.keys()),
                y=list(google_time_line_dic.values()),
                name = "Google Time Line",
                line = dict(color = '#2cd5e8'),
                opacity = 0.8)
microsoft_time_line = go.Scatter(
                x=list(microsoft_time_line_dic.keys()),
                y=list(microsoft_time_line_dic.values()),
                name = "Microsoft Time Line",
                line = dict(color = '#3513a5'),
                opacity = 0.8)
netflix_time_line = go.Scatter(
                x=list(netflix_time_line_dic.keys()),
                y=list(netflix_time_line_dic.values()),
                name = "Netflix Time Line",
                line = dict(color = '#a5127b'),
                opacity = 0.8)

#
data = [amazon_time_line,apple_time_line,facebook_time_line,google_time_line,microsoft_time_line,netflix_time_line]

layout = dict(
    title = "The Overall Time / Numbers of Rewive Chart",
    yaxis = dict(range = [0,80]),
    xaxis = dict(range = ['2008-01-01','2019-01-01'])
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = "The Overall Time / Numbers of Rewive Chart")

In [70]:
layout = dict(
    title = "Amazon Time / Numbers of Rewive Chart",
    yaxis = dict(range = [0,80]),
    xaxis = dict(range = ['2008-01-01','2019-01-01'])
)

fig = dict(data=[amazon_time_line], layout=layout)
py.iplot(fig, filename = "Amazon Time / Numbers of Rewive Chart")

PlotlyRequestError: Hi there! Accounts on the Community Plan cannot create folders.

To save this file on Plotly, please remove the folder in your filename path.

If you would like to create folders and save files in them, please upgrade your account at: https://plot.ly/products/cloud 